In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import outils as ou
import pingouin as pg

In [2]:
home_rel_path = './Public/sim_results.csv'
f_path = ou.get_pure_path(home_rel_path)
dtypes = {'id':'category',
         'model':'category',
         'instock_p':'float64',
         'total_cost':'float64'}
data = ou.csv_to_df(f_path, dtypes)                                  

In [3]:
ids = data['id'].unique()
kpis = ['instock_p', 'total_cost']
models = ['proposed', 'traditional']

In [4]:
observations = {}
for id in ids:
    observations.update({id:{'instock_p':{},'total_cost':{}}})

In [5]:
for id in ids:
    for kpi in kpis:
        for model in models:
            obs_values = ou.get_observations(data, id, model, kpi)
            observations[id][kpi].update({model:obs_values})

In [15]:
mwu_tests_instock_p = []

for id in ids:
    mwu_test = pg.mwu(observations[id]['instock_p']['proposed'], observations[id]['instock_p']['traditional'], tail='greater')
    mwu_test = mwu_test.to_dict(orient='records')[0]
    mwu_test.update({'id':id})
    mwu_tests_instock_p.append(mwu_test)

mwu_tests_instock_p = pd.DataFrame.from_dict(mwu_tests_instock_p)
mwu_tests_instock_p['H0 rejected'] = mwu_tests_instock_p['p-val']<0.025
mwu_tests_instock_p = mwu_tests_instock_p[['id','U-val', 'tail', 'p-val', 'H0 rejected', 'RBC', 'CLES']]

p10 = (mwu_tests_instock_p['H0 rejected'].sum())/len(mwu_tests_instock_p['id'].unique())
print(f'% of null hypothesis rejected: {p7:0.2%}')


% of null hypothesis rejected: 70.97%


In [16]:
mwu_tests_total_cost = []

for id in ids:
    mwu_test = pg.mwu(observations[id]['total_cost']['traditional'], observations[id]['total_cost']['proposed'], tail='greater')
    mwu_test = mwu_test.to_dict(orient='records')[0]
    mwu_test.update({'id':id})
    mwu_tests_total_cost.append(mwu_test)

mwu_tests_total_cost = pd.DataFrame.from_dict(mwu_tests_total_cost)
mwu_tests_total_cost['H0 rejected'] = mwu_tests_total_cost['p-val']<0.05
mwu_tests_total_cost = mwu_tests_total_cost[['id','U-val', 'tail', 'p-val', 'H0 rejected', 'RBC', 'CLES']]

p7 = (mwu_tests_total_cost['H0 rejected'].sum())/len(mwu_tests_total_cost['id'].unique())
print(f'% of null hypothesis rejected: {p7:0.2%}')

% of null hypothesis rejected: 74.19%


In [17]:
mwu_instock_p_H0_result = mwu_tests_instock_p[['id','H0 rejected']]
mwu_instock_p_H0_result.rename(columns={'H0 rejected':'H0 rejected instock_p'}, inplace=True)
mwu_total_cost_H0_result = mwu_tests_total_cost[['H0 rejected']]
mwu_total_cost_H0_result.rename(columns={'H0 rejected':'H0 rejected total_cost'}, inplace=True)
mwu_merged_H0_results = pd.concat([mwu_instock_p_H0_result, mwu_total_cost_H0_result], axis=1, join='inner')
mwu_merged_H0_results['both H0 rejected'] = mwu_merged_H0_results['H0 rejected instock_p'] & mwu_merged_H0_results['H0 rejected total_cost']
p8 = (mwu_merged_H0_results['both H0 rejected'].sum())/len(mwu_merged_H0_results['id'].unique())
print(f'% of null hypothesis rejected: {p8:0.2%}')
mwu_merged_H0_results

% of null hypothesis rejected: 74.19%


,id,H0 rejected instock_p,H0 rejected total_cost,both H0 rejected
0,1,True,False,False
1,2,True,True,True
2,3,True,True,True
3,4,True,True,True
4,5,False,False,False
5,6,True,True,True
6,7,False,False,False
7,8,True,True,True
8,9,False,False,False
9,10,False,False,False


In [9]:
instock95 = np.ones(50)*95

mwu_tests_instock_p_greater95_proposed = []
for id in ids:
    mwu_test = pg.mwu(observations[id]['instock_p']['proposed'], instock95, tail='greater')
    mwu_test = mwu_test.to_dict(orient='records')[0]
    mwu_test.update({'id':id})
    mwu_tests_instock_p_greater95_proposed.append(mwu_test)
    
mwu_tests_instock_p_greater95_proposed = pd.DataFrame.from_dict(mwu_tests_instock_p_greater95_proposed)
mwu_tests_instock_p_greater95_proposed['H0 rejected'] = mwu_tests_instock_p_greater95_proposed['p-val']<0.025
mwu_tests_instock_p_greater95_proposed = mwu_tests_instock_p_greater95_proposed[['id','U-val', 'tail', 'p-val', 'H0 rejected', 'RBC', 'CLES']]
mwu_tests_instock_p_greater95_proposed
p = (mwu_tests_instock_p_greater95_proposed['H0 rejected'].sum())/len(mwu_tests_instock_p_greater95_proposed['id'].unique())
print(f'% of null hypothesis rejected: {p:0.2%}') 

% of null hypothesis rejected: 90.32%


In [13]:
instock95 = np.ones(50)*95

mwu_tests_instock_p_greater95_traditional = []
for id in ids:
    mwu_test = pg.mwu(observations[id]['instock_p']['traditional'], instock95, tail='greater')
    mwu_test = mwu_test.to_dict(orient='records')[0]
    mwu_test.update({'id':id})
    mwu_tests_instock_p_greater95_traditional.append(mwu_test)
    
mwu_tests_instock_p_greater95_traditional = pd.DataFrame.from_dict(mwu_tests_instock_p_greater95_traditional)
mwu_tests_instock_p_greater95_traditional['H0 rejected'] = mwu_tests_instock_p_greater95_traditional['p-val']<0.025
mwu_tests_instock_p_greater95_traditional = mwu_tests_instock_p_greater95_traditional[['id','U-val', 'tail', 'p-val', 'H0 rejected', 'RBC', 'CLES']]
mwu_tests_instock_p_greater95_traditional
p = (mwu_tests_instock_p_greater95_traditional['H0 rejected'].sum())/len(mwu_tests_instock_p_greater95_traditional['id'].unique())
print(f'% of null hypothesis rejected: {p:0.2%}')

% of null hypothesis rejected: 70.97%
